# Excercise: transient channels

This notebook contains an excercise to help you understand transient channels. 

* We will be using the channel toy (see previous lessons) to look at the behaviour of a river in response to changing uplift. 
* We then will look at how this modifies the chi profile. 
* We will then look at a real channel network and let you draw some conclusions about the history of uplift in that location. 

## Before we start: install and import some stuff

In [2]:
!pip install channeltoy ipympl

In [32]:
%matplotlib widget
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interactive
import channeltoy as ct
import matplotlib.pyplot as plt
import numpy as np

## Part 1: Simple channel profile

Lets create an initial channel and then change the uplift rate. Simply execute (shift-enter) the following cell and then play around with `K` and the `initial_U` (which is initial uplift in m/yr) to see how steep a channel is.  

In [57]:
plt.rcParams['figure.figsize'] = [10, 5]

def plot_channel(K = 0.00005, intitial_U = 0.0002, new_U = 0.001 ):
    """Remove old lines from plot and plot new one"""
    #[l.remove() for l in ax.lines]
    
    chan = ct.channeltoy(spacing=250, U = intitial_U, K = K, n=1, m= 0.45)
    initial_elevation = chan.solve_steady_state_elevation()
    x = chan.x_data
    chi = chan.chi_data
    
    #print(x)
    #print(initial_elevation)
    
    plt.plot(x, initial_elevation,label="Initial elevation")
    plt.xlabel("Distance from outlet (m)")
    plt.ylabel("Elevation (m)")
    plt.legend()
     
interactive_plot = interactive(plot_channel, 
                               K=(0.000001, 0.0001, 0.000005), 
                               U=(0.0001, 0.001, 0.0001),
                               new_U=(0.0001, 0.001, 0.0001))

output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=5e-05, description='K', max=0.0001, min=1e-06, step=5e-06), FloatSlide…

## Part 2: A transient channel

This next bit of code sets up an interactive transient channel. 

* It starts with a steady-state channel profile.
* You then increase the erosion rate.
* A knickpoint will develop and will move upslope as you increase the end time.
* Play around a bit to see how fast it moves!

Again, you don't need to adjust anything in the code. Just click on the box and then shift-enter and it will give you an interactive plot. 
Under the hood is a numerical model so you will need to wait a little while each time you change a parameter. 

In [67]:
def plot_channel(K = 0.00005, intitial_U = 0.0002, new_U = 0.0005 ,end_time = 50000):
    """Remove old lines from plot and plot new one"""
    #[l.remove() for l in ax.lines]
    
    chan = ct.channeltoy(spacing=250, U = intitial_U, K = K, n=1, m= 0.45)
    initial_elevation = chan.solve_steady_state_elevation()
    x = chan.x_data
    chi = chan.chi_data
    
    # change the uplift rate
    chan.set_U_values(U = new_U)
    
    times, elevations = chan.transient_simulation(base_level = 0, dt = 200, 
                                              start_time = 0, end_time = end_time+1, 
                                              print_interval = end_time)
    
    #print(times)
    #print(elevations)
    
    plt.plot(x, initial_elevation,label="Initial elevation")
    plt.plot(x, elevations[-1],label = "Time is: "+str(times[-1]))
    plt.xlabel("Distance from outlet (m)")
    plt.ylabel("Elevation (m)")
    plt.legend()
     
interactive_plot = interactive(plot_channel, 
                               K=(0.000001, 0.0001, 0.000005), 
                               U=(0.0001, 0.001, 0.0001),
                               new_U=(0.0001, 0.001, 0.0001),
                               end_time =(10000, 500000, 10000))

output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(FloatSlider(value=5e-05, description='K', max=0.0001, min=1e-06, step=5e-06), FloatSlide…

## Part 3: A transient channel in chi-elevation space

We have explained in class and in some of the previous lessons that slope-area data is has been used to see where channel steepness changes. But this kind of data can be quite noisy. See Lessons 5 and 6. It is easier to see where the knickpoint is using a chi transformation. 

The chi transformation basically squashes the channel at large drainage areas and stretches it at small drainage areas so that, in a steady state landscape, the chi-elevation profile becomes a straight line. 

In chi ($\chi$)-elevation space, the steeper the profile, the higher the steepness index. 

Use the chi profiles below to see where the knickpoint is. Hopefully you can see why this is a little bit easier than using the profiles. 

In [90]:
def plot_channel_chi(K = 0.00005, intitial_U = 0.0002, new_U = 0.0005 ,end_time = 50000):
    """Remove old lines from plot and plot new one"""
    #[l.remove() for l in ax.lines]
    
    chan = ct.channeltoy(spacing=100, U = intitial_U, K = K, n=1, m= 0.45)
    initial_elevation = chan.solve_steady_state_elevation()
    x = chan.x_data
    chi = chan.chi_data
    
    # change the uplift rate
    chan.set_U_values(U = new_U)
    
    times, elevations = chan.transient_simulation(base_level = 0, dt = 200, 
                                              start_time = 0, end_time = end_time+1, 
                                              print_interval = end_time)
    
    plt.plot(chi, initial_elevation,label="Initial elevation")
    plt.plot(chi, elevations[-1],label = "Time is: "+str(times[-1]))
    plt.xlabel("Chi ($\chi$) (m)")
    plt.ylabel("Elevation (m)")
    plt.legend()
    
    return plt.gca()
     
interactive_plot = interactive(plot_channel_chi, 
                               K=(0.000001, 0.0001, 0.000005), 
                               U=(0.0001, 0.001, 0.0001),
                               new_U=(0.0001, 0.001, 0.0001),
                               end_time =(10000, 500000, 10000))

output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(FloatSlider(value=5e-05, description='K', max=0.0001, min=1e-06, step=5e-06), FloatSlide…

## Your assigment

Part 1: Tuning K and U:
* You will need a combination of Google Earth and this website: https://earth.uow.edu.au/ 
* On the https://earth.uow.edu.au/ website, click on the "CRN international river basins" layer. 
    * Find a mountain range you like that has a basin. 
    * Click on the basin: it will have an erosion rate in mm/kyr. Divide by 1000000 to get the erosion rate in m/yr. So 1000 mm/yr will be 0.001 m/yr. 
* Now go onto google earth and find the distance from the mountain front (where the slopes get steep, not the plains) to the peaks, and write down the relief. 